In [342]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [343]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [344]:
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [345]:
%%capture pwd
!pwd

In [346]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('scottish-independence-what-are-the-big-economic-questions', 300, 500)

# Fig 1

In [347]:
# df
df = pd.read_html(
    "https://en.wikipedia.org/wiki/2014_Scottish_independence_referendum"
)[10]
df = df[["Local authority [437]", "Yes (%)"]]
df.columns = ["id", "yes"]
df["yes"] = df["yes"].str.replace("%", "").astype(float)
df["no"] = 100 - df["yes"]
df["diff"] = df["yes"] - 50

# topojson
# https://martinjc.github.io/UK-GeoJSON/json/sco/topo_lad.json simplyfiy on mapshaper.org
scot = json.loads(open("visualisation/fig1/scotland_lad_mini.json", "r").read())

In [348]:
# ### geojson style, no necessary now

# load topojson
# https://martinjc.github.io/UK-GeoJSON/json/sco/topo_lad.json simplyfiy on mapshaper.org

for i in scot["objects"]["lad"]["geometries"]:
    lad = (
        i["properties"]["LAD13NM"]
        .replace(" City", "")
        .replace("City of ", "")
        .replace(" Islands", "")
    )
    i["properties"]["LAD13NM"] = lad
    try:
        for k in df.set_index("id").columns:
            i["properties"][k] = df.set_index("id").loc[lad][k]
    except:
        print(lad)

open("visualisation/fig1/scotland_lad_mini_data.json", "w").write(json.dumps(scot))

135249

In [349]:
open("visualisation/fig1/2014_referendum.json", "w").write(
    json.dumps(list(df.T.to_dict().values()))
)

2457

In [350]:
readme = (
    "### fig1_2014_referendum"
    + "\n[Interactive map](https://economicsobservatory.github.io/ECOvisualisations/articles/scottish-independence-what-are-the-big-economic-questions/visualisation/fig1/)\n\n"
)
open("README.md", "w").write(readme)

189

# Fig 2

In [351]:
df = pd.read_csv("raw/poll_data.csv")
df["Fieldwork End Date / Pollster"] = (
    df["Fieldwork End Date / Pollster"].str.split("/").str[0]
)
df.columns = ["date", "yes", "no"]
for k in ["yes", "no"]:
    df[k] = df[k].str.replace("%", "").astype(float)
df["date"] = pd.to_datetime(df["date"]).astype(str)

In [352]:
f = "fig2_polls"
f2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,date,yes,no
0,2016-06-25,53.0,47.0
1,2016-06-26,52.0,48.0
2,2016-06-28,53.0,47.0
3,2016-07-25,47.0,53.0
4,2016-08-31,47.0,53.0


In [355]:
base = alt.Chart(f2).encode(
    x=alt.X(
        "date:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
)
line1 = base.mark_line(color=colors["eco-mid-blue"]).encode(
    y=alt.Y(
        "yes:Q",
        sort=[],
        axis=alt.Axis(
#             grid=False,
            gridOpacity=0.3,
            title="%",
            titleX=5,
            titleY=5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
        scale=alt.Scale(domain=[30, 70]),
    )
)
line2 = base.mark_line(color=colors["eco-dot"]).encode(y=alt.Y("no:Q"))
area1 = base.mark_area(
    interpolate="monotone",
    fillOpacity=0.7,
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color=colors["eco-dot"], offset=0.7),
            alt.GradientStop(color="white", offset=0.5),
            alt.GradientStop(color=colors["eco-mid-blue"], offset=0.3),
        ],
        x1=1,
        x2=1,
        y1=1,
        y2=0,
    ),
).encode(y="no:Q", y2="yes:Q")
area1a = area1.transform_filter("datum.date<toDate('2020-01-01')")
area1b = area1.transform_filter("datum.date>toDate('2021-03-01')")
area2 = (
    base.mark_area(
        interpolate="monotone",
        fillOpacity=0.7,
        color=alt.Gradient(
            gradient="linear",
            stops=[
                alt.GradientStop(color=colors["eco-dot"], offset=0.3),
                alt.GradientStop(color="white", offset=0.5),
                alt.GradientStop(color=colors["eco-mid-blue"], offset=0.7),
            ],
            x1=1,
            x2=1,
            y1=1,
            y2=0,
        ),
    )
    .encode(y="no:Q", y2="yes:Q")
    .transform_filter("datum.date>toDate('2020-01-01')")
    .transform_filter("datum.date<toDate('2021-03-01')")
)
label1 = (
    alt.Chart(pd.DataFrame([{"date": "2017-07-01", "y": 60.5, "t": "No"}]))
    .mark_text(color=colors["eco-dot"], align="left")
    .encode(x="date:T", y="y:Q", text="t:N")
)
label2 = (
    alt.Chart(pd.DataFrame([{"date": "2017-07-01", "y": 39.5, "t": "Yes"}]))
    .mark_text(color=colors["eco-mid-blue"], align="left")
    .encode(x="date:T", y="y:Q", text="t:N")
)
layer1 = (
    (
        (line1 + line2 + area1a + area1b + area2 + label1 + label2).properties(
            height=300, width=400
        )
    )
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

# Fig 4

In [356]:
df = pd.read_excel("raw/UK_Public_Spending_Data.xlsx", sheet_name="9_1b", skiprows=3)
df = df[["Unnamed: 0", "2019-20\noutturn.1"]]
df.columns = ["region", "index"]
df=df.dropna().iloc[:-1]

In [357]:
f = "fig4_public_spending"
f4 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f4.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f4 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,region,index
0,North East,104.0
1,North West,103.0
2,Yorkshire & The Humber,95.0
3,East Midlands,90.0
4,West Midlands,97.0


In [358]:
base = alt.Chart(f4).encode(
    x=alt.X(
        "region:N",
        axis=None
    )
)
bars = base.mark_bar(color=colors["eco-dot"],opacity=0.9).encode(
    y=alt.Y(
        "index:Q",
        sort=[],
        axis=alt.Axis(
#             grid=False,
            gridOpacity=0.3,
            title="Public spending index 2019-20| UK average = 100",
            titleX=-25,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        )
    )
)
scot = base.mark_bar(color=colors["eco-mid-blue"],opacity=0.9).encode(
    y=alt.Y(
        "index:Q",
        sort=[]
    )
).transform_filter("datum.region=='Scotland'")
axis1 = (
    alt.Chart(pd.DataFrame([{"x": "East", "y": 100}, {"x": "Yorkshire & The Humber", "y":100}]))
    .mark_line(strokeWidth=1, strokeDash=[5, 5], color=colors["eco-gray"])
    .encode(x=alt.X("x:N"), y="y:Q")
)
text=bars.mark_text(color='white',angle=270,align='left',dx=5).encode(text='region:N',y='y:Q').transform_calculate(y='0')
layer1 = (
    ((bars+scot+text+axis1).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

# Fig 5

In [359]:
# df
df = pd.read_csv('raw/mye-2020-council-change-custom.csv')
df=df[['Area','Percentage_Change']]
df['Percentage_Change']=df['Percentage_Change'].str.replace('%','').astype(float)
df['Area']=df['Area'].str.replace(" City", "").str.replace("City of ", "").str.replace(" Islands", "").replace('Na h-Eileanan Siar','Eilean Siar')
df.columns=['id','pop']

In [360]:
open("visualisation/fig5/population_change.json", "w").write(
    json.dumps(list(df.T.to_dict().values()))
)

1223

In [361]:
readme = (
    "### fig5_population_change"
    + "\n[Interactive map](https://economicsobservatory.github.io/ECOvisualisations/articles/scottish-independence-what-are-the-big-economic-questions/visualisation/fig5/)\n\n"
)
open("README.md", "a").write(readme)

191

# Fig 6

## a

In [362]:
df = pd.read_excel("raw/Industry_mix.xlsx")
df.columns=['Industry','LQ','UK']

In [363]:
f = "fig6a_industry_mix"
f6a = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f6a.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f6a = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,Industry,LQ,UK
0,"Agriculture, forestry and fishing",2.031258,1
1,Mining and quarrying,7.051976,1
2,Manufacturing,0.954301,1
3,"Electricity, gas, etc",1.652874,1
4,"Water supply; sewerage, waste management etc",1.096523,1


In [364]:
base = alt.Chart(f6a).encode(
    x=alt.X(
        "Industry:N",
        axis=None
    )
)
bars = base.mark_bar(color=colors["eco-dot"],opacity=0.9).encode(
    color=alt.Color('Industry:N',legend=None),
    y=alt.Y(
        "LQ:Q",
        sort=[],
        axis=alt.Axis(
#             grid=False,
            gridOpacity=0.3,
            title=["Location quotient (LQ)",'Industry share of employment in Scotland','is the same as it is in the UK, when LQ=1'],
            titleX=5,
            titleY=5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        )
    )
)
axis1 = (
    base
    .mark_line(strokeWidth=1, strokeDash=[5, 5], color=colors["eco-gray"])
    .encode(x="Industry:N", y="UK:Q")
)
text1=bars.mark_text(angle=270,align='left',dx=5).encode(text='Industry:N').transform_filter("datum.Industry!='Mining and quarrying'")
text2=base.mark_text(angle=270,align='right',dx=-5,color='white').encode(x='Industry:N',y='LQ:Q',text='Industry:N').transform_filter("datum.Industry=='Mining and quarrying'")
layer1 = (
    ((bars+text1+text2+axis1).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

## b